In [1]:
import tensorflow as tf
import keras
import numpy as np

In [2]:
# Residual connection : Used so that the noise to the input that occurs during back propagation is minimized
# We add a residual connection so that the input of the layer is added to the output of the layer( the shape of the output and input should be the same)

In [3]:
#x ------> input
#residual = x
#x = block(x) # block in the layer
#x = add([x,residual]) # adding the input and the output of the layer(the output will have the full information of the input)

In [4]:
# However , in convolutional layers, the shapes of the input and the output is different, if the number of filters is increased or max pooling is introduced
# We use a 1 X 1 Conv2D layer with no activation to add the input and the output of the layer (padding = "same" required for this)
# If max pooling is introduced, then if max pooling = 2 , then next 1X1 conv2D layer will have strides=2 to match the downsampling caused by the max pooling, and then the residual is added


In [12]:
from keras.layers import Input,Conv2D,MaxPool2D,add


In [16]:
# Applying residual connections to the convolutional block
input_layer = Input(shape=(32,32,3))

conv_layer1 = Conv2D(32,3,activation="relu")(input_layer)
residual_1 = conv_layer1 # Output of conv_layer 1 , hence input of conv_layer2

#Conv block around which we create the residuals
#For conv_layer 2 , layer is defined along with max pooling 
conv_layer2 = Conv2D(64,3,activation="relu",padding="same")(conv_layer1)
max_pool1 = MaxPool2D(2,padding="same")(conv_layer2)

# residual_2 is the 1X1 layer with no activation(64 is the number of kernels that will be similar to the previous conv layer), strides=2 since max pooling = 2, and its input will be the previous residual(because we are converting the input into the size of the output of the conv layer 2)
residual_2 = Conv2D(64,1,strides=2)(residual_1) 
add_layer = add([max_pool1,residual_2]) # Adding the previous formatted input with the output of the max pool layer i.e the conv block output of conv_layer2


In [17]:
from keras.layers import Rescaling

In [40]:
# Defining a function which creates a residual block with 2 convolutional layers and depending on the pooling parameter, it will have strides or not

input_layer = Input(shape=(32,32,3))
rescale_layer = Rescaling(1./255)(input_layer)

def build_residual_block(previous_layer,filters,pool_size=0,pooling=False): # Pooling default value as False
    residual = previous_layer
    conv_layer1 = Conv2D(filters,3,activation="relu",padding="same")(previous_layer)
    conv_layer2 = Conv2D(filters,3,activation="relu",padding="same")(conv_layer1)
    if pooling == False:
        residual_reshape = Conv2D(filters,1,padding="same")(residual)
        add_layer = add([conv_layer2,residual_reshape])
        return add_layer
    elif pooling==True:
        max_pool1 = MaxPool2D(pool_size,padding="same")(conv_layer2)
        residual_reshape = Conv2D(filters,1,strides=pool_size)(residual)
        add_layer = add([max_pool1,residual_reshape])
        return add_layer


In [45]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

In [48]:
res_block1 = build_residual_block(rescale_layer, filters=64, pool_size=2, pooling=True)
res_block2 = build_residual_block(res_block1, filters=128, pool_size=2, pooling=False)

# Apply Global Average Pooling
gap_layer = GlobalAveragePooling2D()(res_block2) # Used as an alternative to flatten() : averages the values of each channel across the spatial dimensions (width and height) of the feature maps, resulting in a single value per channel.

# Add a dense layer
output_layer = Dense(1, activation='softmax')(gap_layer)

# Define the model
model = Model(inputs=input_layer, outputs=output_layer)

# Display model summary
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 rescaling_4 (Rescaling)        (None, 32, 32, 3)    0           ['input_14[0][0]']               
                                                                                                  
 conv2d_59 (Conv2D)             (None, 32, 32, 64)   1792        ['rescaling_4[0][0]']            
                                                                                                  
 conv2d_60 (Conv2D)             (None, 32, 32, 64)   36928       ['conv2d_59[0][0]']              
                                                                                            

In [49]:
# Batch normalization : Used for normalization of outputs of the convolutional layers
# If possible, try to apply normalization before applying activation like below

In [51]:
from keras.layers import BatchNormalization, Activation

In [53]:
# One convolution layer
#conv_layer = Conv2D(32,3,use_bias=False)
#conv_layer = BatchNormalization()(conv_layer)
#conv_layer = Activation("relu")(conv_layer)